In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/legal-text-classification-dataset/legal_text_classification.csv
/kaggle/input/legalsummary/HFsummarydataset.csv


In [2]:
!pip install llama-index pinecone sentence-transformers
!pip install llama-index-vector-stores-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 7.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 42.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.6/250.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requi

In [3]:
from pinecone import Pinecone, ServerlessSpec
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    SimpleDirectoryReader,
)
from llama_index.vector_stores.pinecone import PineconeVectorStore

# Initialize Pinecone
pc = Pinecone(api_key="pcsk_7E1vP3_QK2W6M9XCiHufegYYXyYurwWGddethJCY4TYTcm15PnRKS5EDMKa11KyYvNjVbB")
pc_duplicate = Pinecone(api_key="pcsk_7VMsMN_PRA9cnkzr5tpG92dfxqxcAhhczX6RwifiDqkCTa4ruyWZX5AXyJNgiCezztAgvT")
query_pc = Pinecone(api_key="pcsk_6zNLd7_Lep8rBqzNatyGXGykxBumtiAcYE493jHtrgTkjhuJYdDBGeByH8KXAFYVdXLjrq")

# Create a Pinecone index
index_name = "legaltextcasesummary"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=1024, 
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )




# NO Q/A dataset
Data Preprocessing to fillna and then combine all the columns

In [ ]:
# # Load the CSV file and process its content
# csv_file_path = "/kaggle/input/legal-text-classification-dataset/legal_text_classification.csv"
# df = pd.read_csv(csv_file_path)
# # Handle missing values by filling them with empty strings
# df['case_title'] = df['case_title'].fillna("")
# df['case_outcome'] = df['case_outcome'].fillna("")
# df['case_text'] = df['case_text'].fillna("")

# # Combine columns into a single text for indexing
# df['content'] = (
#     "Case Title: " + df['case_title'] + "\n" +
#     "Case Outcome: " + df['case_outcome'] + "\n" +
#     "Case Text: " + df['case_text']
# )
# df

In [ ]:
# content = list(df['content'])
# ids = list(df['case_id'])
# data = []
# for i in range(len(ids)):
#     data.append({'id':ids[i],'text':content[i]})

In [ ]:

# data[0]['text']

Inputting/Upserting to Pincecone DB 

In [ ]:
# import time
# for i in range(96,len(data)-96,96):
#     datas = data[i:i+96]
#     embeddings = pc.inference.embed(
#         model="multilingual-e5-large",
#         inputs=[d['text'] for d in datas],
#         parameters={"input_type": "passage", "truncate": "END"}
#     )
#     pinecone_index = pc.Index(index_name)
#     records = []
#     for d, e in zip(datas, embeddings):
#         records.append({
#             "id": d['id'],
#             "values": e['values'],
#             "metadata": {'text': d['text']}
#         })
    
#     # Upsert the records into the index
#     pinecone_index.upsert(
#         vectors=records,
#         namespace=f"ind_{i}"
#     )
#     time.sleep(60)

# HF dataset with summary

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# df = load_dataset("ninadn/indian-legal")
# df

In [ ]:
# train = pd.DataFrame(df['train'])
# train_text = list(train['Text'])
# train_summary = list(train['Summary'])

In [ ]:
# train_text[3]

In [ ]:
# import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize

# nltk.download("punkt_tab")  # Download NLTK tokenizer model

In [ ]:
# def chunk_text(text, max_tokens=505):
#     """Splits text into chunks of max 505 tokens while keeping sentence structure intact."""
#     sentences = sent_tokenize(text)  # Split into sentences
#     chunks = []
#     current_chunk = []
#     current_length = 0

#     for sentence in sentences:
#         words = word_tokenize(sentence)  # Tokenize words
#         sentence_length = len(words)

#         if current_length + sentence_length > max_tokens:
#             if current_chunk:
#                 chunks.append(" ".join(current_chunk))  # Save previous chunk
#             current_chunk = words  # Start new chunk
#             current_length = sentence_length
#         else:
#             current_chunk.extend(words)
#             current_length += sentence_length

#     if current_chunk:
#         chunks.append(" ".join(current_chunk))  # Add last chunk

#     return chunks

In [ ]:
df = pd.read_csv('/kaggle/input/legalsummary/HFsummarydataset.csv')
df

In [ ]:
data = []
chunks_text = list(df['text'])
summary_text = list(df['summary'])
ids = list(df['id'])


In [ ]:
for i in range(len(ids)):    
    data.append({'id':ids[i],'text':chunks_text[i],'summary':summary_text[i]})

In [ ]:
# data = []
# for i in range(len(train_text)):
#     chunks = chunk_text(train_text[i])
#     for j in range(len(chunks)):
#         data.append({'id':'ns_'+str(i)+'_'+str(j),'text':chunks[j],'summary':train_summary[i]})

In [ ]:
data[0]['summary']

# upsert HF dataset to Pinecone

In [ ]:
# import time

# # Define a function to truncate text to fit metadata limits
# def truncate_text(text, max_bytes=40960):
#     """Truncate text to ensure metadata size doesn't exceed the limit."""
#     while len(text.encode('utf-8')) > max_bytes:
#         text = text[:-100]  # Remove 100 characters at a time
#     return text
# namespace_count=35
# # Process data in batches
# for i in range(61824, len(data) - 192, 192):
#     if i%1920==0:
#         namespace_count+=1
#     try:
#         datas = []
#         embeddings = []
#         for j in range(1,3):
#             # Slice the data batch
#             s = i+(j-1)*96
#             e = i+j*96
#             datas.append(data[s:e])
#             # Generate embeddings
#             emb = pc_duplicate.inference.embed(
#                 model="multilingual-e5-large",
#                 inputs=[d['text'] for d in datas[j-1]],
#                 parameters={"input_type": "passage", "truncate": "END"}
#             )
#             embeddings.append(emb) 
#             time.sleep(15)
#         # Prepare records for Pinecone
#         records = []
#         for j in range(len(datas)):
#             for d, e in zip(datas[j], embeddings[j]):
#                 try:
#                     # # Truncate text if metadata exceeds the limit
#                     truncated_text = d['text']
#                     summary_text = d['summary']
                    
#                     # Add the record
#                     records.append({
#                         "id": d['id'],
#                         "values": e['values'],
#                         "metadata": {"text": truncated_text,'summary':summary_text}
#                     })
#                 except Exception as inner_error:
#                     print(f"Error processing record ID {d['id']}: {inner_error}")
            
#         # Upsert records into the Pinecone index
#         pinecone_index = pc.Index(index_name)
#         pinecone_index.upsert(
#             vectors=records,
#             namespace=f"ind_{namespace_count}"
#         )
#     except Exception as batch_error:
#         print(f"Error processing batch {i}: {batch_error}")


In [5]:
# # Create a Pinecone index
# q_index_name = "legaltextcasequery"
# if index_name not in pc_duplicate.list_indexes().names():
#     pc_duplicate.create_index(
#         name=q_index_name, 
#         dimension=1024, 
#         metric='cosine',
#         spec=ServerlessSpec(
#             cloud='aws',
#             region='us-east-1'
#         )
#     )

In [6]:
# # Define your query
# query = "What is the section that sets up machinery to assess tax liabilities of firms which have discontinued their business?"

# # Convert the query into a numerical vector that Pinecone can search with
# query_embedding = pc_duplicate.inference.embed(
#     model="multilingual-e5-large",
#     inputs=[query],
#     parameters={
#         "input_type": "query"
#     }
# )

In [13]:
# q_records = []
# q_results = pc_duplicate.Index(q_index_name).query(
#     namespace="q_ind",
#     vector=query_embedding[0].values,
#     top_k=3,
#     include_values=False,
#     include_metadata=True 
# )
# if q_results['matches']:
#     print('Not empty')
# else:
#     print('empty')
    
    

empty


In [ ]:
# # Search the index for the three most similar vectors
# results = pc.Index(index_name).query(
#     namespace="ind_1",
#     vector=query_embedding[0].values,
#     top_k=3,
#     include_values=False,
#     include_metadata=True
# )

# print(results)

{'matches': [{'id': 'ns_321_11',
              'metadata': {'summary': 'Two brothers who had been carrying on '
                                      'in partnership a business, which had '
                                      'been assessed to income tax under the '
                                      'Indian Income tax Act of 1918 and the '
                                      'accounting year of which was a period '
                                      'of 12 months ending on the 30th June '
                                      'each year, transferred the business to '
                                      'a limited company on the 1st March, '
                                      '1940, and claimed in the assessment for '
                                      'the year 1940 41 that under section 25 '
                                      '(4) of the Income tax Act, 1922, they '
                                      'were not liable to pay income tax on '
                    

In [6]:
def isSimilarQuery(query):
    # Create a Pinecone index
    q_index_name = "legaltextcasequery"
    if q_index_name not in query_pc.list_indexes().names():
        pc_duplicate.create_index(
            name=q_index_name, 
            dimension=1024, 
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )
    # Convert the query into a numerical vector that Pinecone can search with
    query_embedding = pc_duplicate.inference.embed(
        model="multilingual-e5-large",
        inputs=[query],
        parameters={
            "input_type": "query"
        }
    )
    q_results = query_pc.Index(q_index_name).query(
        namespace="q_ind",
        vector=query_embedding[0].values,
        top_k=3,
        include_values=False,
        include_metadata=True 
    )
    q_records=[]
    if q_results['matches'] and q_results['matches'][0]['score']>=0.8:
        print('Query Present')
        return q_results
    else:
        # Search the index for the three most similar vectors
        results = pc.Index(index_name).query(
            namespace="ind_1",
            vector=query_embedding[0].values,
            top_k=3,
            include_values=False,
            include_metadata=True
        )
        q_results= results
        for i in range(3):
            q_results['matches'][i]['metadata']['query'] = query
            q_records.append({
                "id":q_results['matches'][i]['id'],
                "values":query_embedding[0].values,
                "metadata":q_results['matches'][i]['metadata'],
            })
        query_pc.Index(q_index_name).upsert(
            vectors=q_records,
            namespace="q_ind",
        )
        print('Query Upserted')
        return q_records
    

In [7]:
query = "What is the section that sets up machinery to assess tax liabilities of firms which have discontinued their business?"
print(isSimilarQuery(query))

Query Present
{'matches': [{'id': 'ns_31_2',
              'metadata': {'query': 'What is the section that sets up '
                                    'machinery to assess tax liabilities of '
                                    'firms which have discontinued their '
                                    'business?',
                           'summary': 'The appellant who was carrying on '
                                      'business in food grains in partnership '
                                      'with another person submitted the '
                                      'returns of the income of the firm for '
                                      'the accounting years even after his '
                                      "partner 's death.\n"
                                      'It was found that certain income of the '
                                      'firm was concealed and the Income tax '
                                      'Officer not only assessed the firm t